In [1]:
### HILBERT LOCK ###
#
# Jon's "tracer bullet" April 14, 2025
#
# jonathan.kaufman@quantumburgh.com
#
# Start with a weak combination lock, and then gradually
# dial down its weakness until its energy landscape vanishes
# everywhere except at a single point: the combination.
# And maybe suppose n-dials-of-k (where k is prime).

In [2]:
from IPython.display import display, Math
from itertools import product

class HLock:
        
    def __init__(self):
        #algebraic group Z_3^5
        self.k = 5 # must be prime if assuming field axioms
        self.n = 3
        self.code = (0,0,0) # structure is isomorphic to any code
        self.degeneracy = [1, 6, 18, 32, 36, 24, 8] # triple self convolution of Z_5
    
    def norm(self, setting: tuple) -> int:
        #Banach space
        #Manhattan distance
        #zero = (0,0,0)
        s = setting # abreviation going forward
        a, b, c = s
        d = lambda x: min(x, self.k-x)
        return d(a) + d(b) + d(c)
    
    def add(self, s1: tuple, s2: tuple) -> tuple:
        #group operation
        #modulo k
        return tuple((x + y) % self.k for x, y in zip(s1, s2))
    
    def neg(self, s: tuple) -> tuple:
        #group inverse
        k = self.k
        a, b, c = s
        return tuple(((k-a)%k, (k-b)%k, (k-c)%k))
    
    def diff(self, s1: tuple, s2: tuple) -> tuple:
        #add a neg
        p1 = s1
        p2 = s2
        n2 = self.neg(p2)
        return self.add(p1,n2)
    
    def metric(self, s1: tuple, s2: tuple) -> int:
        #metric space
        d12 = self.diff(s1, s2)
        return self.norm(d12)
    
    def mult(self, a: int, s: tuple) -> tuple:
        #algebraic ring
        #scalar * vector
        k = self.k
        x, y, z = s
        return tuple(((a*x)%k, (a*y)%k, (a*z)%k))

    def dot(self, s1: tuple, s2: tuple) -> int:
        #pseudo-Hilbert
        #inner (dot) product
        k = self.k
        a, b, c = s1
        x, y, z = s2
        return (((a*x)%k + (b*y)%k + (c*z)%k)%k)
    
    def dual(self, s: tuple) -> tuple:
        #bra (as in bra-ket)
        return s
    
    def weak_field(self, s: tuple) -> int:
        #error (potential energy)
        return self.metric(self.code, s)

    def stratum(self, level):
        #energy level
        k=self.k
        s = set()
        for xyz in product(range(k), repeat=3):
            if self.weak_field(xyz) == level:
                s.add(xyz)
        return s

    def neighbors(self, s: tuple) -> set:
        #Manhattan distance = 1
        ss = set()
        a, b, c = s
        return{(a+1,b,c),(a-1,b,c),(a,b+1,c),(a,b-1,c),(a,b,c+1),(a,b,c-1)}
    
    @classmethod
    def test(cls):
        #method tests
        l = cls() #instance
        p = '\033[92m'+'pass'+'\033[0m'
        f = '\033[91m'+'fail'+'\033[0m'
        print()
        print('testing space: ', end=' ')
        display(Math(r'\mathbb{Z}_5^3'))
        print('norm():       ', p if l.norm((1,2,3))            ==       5  else f)
        print('add():        ', p if l.add((1,2,3), (2,3,1))    == (3,0,4)  else f)
        print('neg():        ', p if l.neg((1,2,3))             == (4,3,2)  else f)
        print('mult():       ', p if l.mult(2, (1,2,3))         == (2,4,1)  else f)
        print('diff():       ', p if l.diff((1,2,3), (2,1,3))   == (4,1,0)  else f)
        print('metric():     ', p if l.metric((1,2,3), (2,2,3)) ==       1  else f)
        print('dot():        ', p if l.dot((0,1,1), (1,2,3))    ==       0  else f)
        print('dual():       ', p if l.dual((1,2,3))            == (1,2,3)  else f)
        print('weak_field(): ', p if l.weak_field((2,2,1))      ==       5  else f)
        print('stratum():    ', p if len(l.stratum(4))          ==      36  else f)
        print('neighbors():  ', p if (0, 0, 3) in l.neighbors((0,0,4))      else f)
        print()

In [3]:
l=HLock()
l.test()


testing space:  

<IPython.core.display.Math object>

norm():        pass
add():         pass
neg():         pass
mult():        pass
diff():        pass
metric():      pass
dot():         pass
dual():        pass
weak_field():  pass
stratum():     pass
neighbors():   pass

